In [1]:
from mpi4py import MPI

ImportError: DLL load failed: Не найден указанный модуль.

In [1]:
import numpy as np
import tensorflow as tf
import gym
import time
import core as core
from logx import EpochLogger
from mpi_tf import MpiAdamOptimizer, sync_all_params
from mpi_tools import mpi_fork, mpi_avg, proc_id, mpi_statistics_scalar, num_procs

from SpiderEnv import SpiderEnv
from SpiderEnv2 import SpiderEnv as SpiderEnv2
from SpiderEnvMany import SpiderEnv as SpiderEnvMany
from SpiderEnvMany2 import SpiderEnv as SpiderEnvMany2

from tqdm import tqdm_notebook
import matplotlib.pyplot as plt

ImportError: DLL load failed: Не найден указанный модуль.

In [2]:
env_name = "SpiderEnvMany2"
env = SpiderEnvMany2()
actor_critic = core.mlp_actor_critic
seed = 0 
steps_per_epoch = 4000
epochs = 5000
gamma = 0.99 
clip_ratio = 0.2 
pi_lr = 3e-4
vf_lr = 1e-3
train_pi_iters = 80 
train_v_iters = 80
lam = 0.97
max_ep_len = 1000
target_kl = 0.01


save_freq = 100
log_info_every_epoch = 10
cpu = 1
exp_name = "ppo"


ac_kwargs=dict(hidden_sizes=[100, 50, 25])

In [3]:
from run_utils import setup_logger_kwargs
logger_kwargs = setup_logger_kwargs(exp_name, seed)

In [4]:
mpi_fork(1) 

In [5]:
plots_data = {
    "pi_loss": [],
    "actor_gradients_max_abs": [],
    "actor_gradients_equal_zero": [],
    "v_loss": [],
    "critic_gradients_max_abs": [],
    "critic_gradients_equal_zero": [],
    "value_mean": [],
    "rewards": [],
    "adv_mean": [],
}

In [6]:
class PPOBuffer:
    """
    A buffer for storing trajectories experienced by a PPO agent interacting
    with the environment, and using Generalized Advantage Estimation (GAE-Lambda)
    for calculating the advantages of state-action pairs.
    """

    def __init__(self, obs_dim, act_dim, size, gamma=0.99, lam=0.95):
        self.obs_buf = np.zeros(core.combined_shape(size, obs_dim), dtype=np.float32)
        self.act_buf = np.zeros(core.combined_shape(size, act_dim), dtype=np.float32)
        self.adv_buf = np.zeros(size, dtype=np.float32)
        self.rew_buf = np.zeros(size, dtype=np.float32)
        self.ret_buf = np.zeros(size, dtype=np.float32)
        self.val_buf = np.zeros(size, dtype=np.float32)
        self.logp_buf = np.zeros(size, dtype=np.float32)
        self.gamma, self.lam = gamma, lam
        self.ptr, self.path_start_idx, self.max_size = 0, 0, size

    def store(self, obs, act, rew, val, logp):
        """
        Append one timestep of agent-environment interaction to the buffer.
        """
        assert self.ptr < self.max_size     # buffer has to have room so you can store
        self.obs_buf[self.ptr] = obs
        self.act_buf[self.ptr] = act
        self.rew_buf[self.ptr] = rew
        self.val_buf[self.ptr] = val
        self.logp_buf[self.ptr] = logp
        self.ptr += 1

    def finish_path(self, last_val=0):
        """
        Call this at the end of a trajectory, or when one gets cut off
        by an epoch ending. This looks back in the buffer to where the
        trajectory started, and uses rewards and value estimates from
        the whole trajectory to compute advantage estimates with GAE-Lambda,
        as well as compute the rewards-to-go for each state, to use as
        the targets for the value function.
        The "last_val" argument should be 0 if the trajectory ended
        because the agent reached a terminal state (died), and otherwise
        should be V(s_T), the value function estimated for the last state.
        This allows us to bootstrap the reward-to-go calculation to account
        for timesteps beyond the arbitrary episode horizon (or epoch cutoff).
        """

        path_slice = slice(self.path_start_idx, self.ptr)
        rews = np.append(self.rew_buf[path_slice], last_val)
        vals = np.append(self.val_buf[path_slice], last_val)
        
        # the next two lines implement GAE-Lambda advantage calculation
        deltas = rews[:-1] + self.gamma * vals[1:] - vals[:-1]
        self.adv_buf[path_slice] = core.discount_cumsum(deltas, self.gamma * self.lam)
        
        # the next line computes rewards-to-go, to be targets for the value function
        self.ret_buf[path_slice] = core.discount_cumsum(rews, self.gamma)[:-1]
        
        self.path_start_idx = self.ptr

    def get(self):
        """
        Call this at the end of an epoch to get all of the data from
        the buffer, with advantages appropriately normalized (shifted to have
        mean zero and std one). Also, resets some pointers in the buffer.
        """
        assert self.ptr == self.max_size    # buffer has to be full before you can get
        self.ptr, self.path_start_idx = 0, 0
        # the next two lines implement the advantage normalization trick
        adv_mean, adv_std = mpi_statistics_scalar(self.adv_buf)
        self.adv_buf = (self.adv_buf - adv_mean) / adv_std
        return [self.obs_buf, self.act_buf, self.adv_buf, 
                self.ret_buf, self.logp_buf]

In [7]:
logger = EpochLogger(**logger_kwargs)
# logger.save_config(locals())

seed += 10000 * proc_id()
tf.set_random_seed(seed)
np.random.seed(seed)

obs_dim = env.observation_space.shape
act_dim = env.action_space.shape

# Share information about action space with policy architecture
ac_kwargs['action_space'] = env.action_space

# # Inputs to computation graph
x_ph, a_ph = core.placeholders_from_spaces(env.observation_space, env.action_space)
adv_ph, ret_ph, logp_old_ph = core.placeholders(None, None, None)

# Main outputs from computation graph
pi, logp, logp_pi, v = actor_critic(x_ph, a_ph, **ac_kwargs)

# Need all placeholders in *this* order later (to zip with data from buffer)
all_phs = [x_ph, a_ph, adv_ph, ret_ph, logp_old_ph]

# Every step, get: action, value, and logprob
get_action_ops = [pi, v, logp_pi]

# Experience buffer
local_steps_per_epoch = int(steps_per_epoch / num_procs())
buf = PPOBuffer(obs_dim, act_dim, local_steps_per_epoch, gamma, lam)

# Count variables
var_counts = tuple(core.count_vars(scope) for scope in ['pi', 'v'])
logger.log('\nNumber of parameters: \t pi: %d, \t v: %d\n'%var_counts)

# PPO objectives
ratio = tf.exp(logp - logp_old_ph)          # pi(a|s) / pi_old(a|s)
min_adv = tf.where(adv_ph>0, (1+clip_ratio)*adv_ph, (1-clip_ratio)*adv_ph)
pi_loss = -tf.reduce_mean(tf.minimum(ratio * adv_ph, min_adv))
v_loss = tf.reduce_mean((ret_ph - v)**2)

# Info (useful to watch during learning)
approx_kl = tf.reduce_mean(logp_old_ph - logp)      # a sample estimate for KL-divergence, easy to compute
approx_ent = tf.reduce_mean(-logp)                  # a sample estimate for entropy, also easy to compute
clipped = tf.logical_or(ratio > (1+clip_ratio), ratio < (1-clip_ratio))
clipfrac = tf.reduce_mean(tf.cast(clipped, tf.float32))

# Optimizers
train_pi = MpiAdamOptimizer(learning_rate=pi_lr).minimize(pi_loss)
train_v = MpiAdamOptimizer(learning_rate=vf_lr).minimize(v_loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

# Sync params across processes
sess.run(sync_all_params())

# Setup model saving
logger.setup_tf_saver(sess, inputs={'x': x_ph}, outputs={'pi': pi, 'v': v})

W0723 20:59:27.009950 140397783583616 deprecation_wrapper.py:119] From /home/suriknik/Projects/sirius/mujoco/core.py:14: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



Logging data to /home/suriknik/Projects/sirius/data/ppo/ppo_s0/progress.txt


W0723 20:59:27.234605 140397783583616 deprecation_wrapper.py:119] From /home/suriknik/Projects/sirius/mujoco/core.py:98: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0723 20:59:27.237413 140397783583616 deprecation.py:323] From /home/suriknik/Projects/sirius/mujoco/core.py:31: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W0723 20:59:27.249476 140397783583616 deprecation.py:506] From /home/suriknik/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0723 20:59:28.531372 140397783583616 deprecation_wrapper.py:119] From /home/suriknik/Projects/sirius/mujoc


Number of parameters: 	 pi: 90111, 	 v: 89651



W0723 20:59:29.759853 140397783583616 deprecation.py:323] From /home/suriknik/Projects/sirius/mujoco/mpi_tf.py:63: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


In [8]:
# saver = tf.train.Saver()
# saver.restore(sess, '/home/sirius/Рабочий стол/models/"Ant-v2"-epoch-700.ckpt.data-00000-of-00001')

In [ ]:
def update():
    inputs = {k:v for k,v in zip(all_phs, buf.get())}
    pi_l_old, v_l_old, ent = sess.run([pi_loss, v_loss, approx_ent], feed_dict=inputs)

    # Training
    for i in range(train_pi_iters):
        _, kl = sess.run([train_pi, approx_kl], feed_dict=inputs)
        kl = mpi_avg(kl)
        if kl > 1.5 * target_kl:
            logger.log('Early stopping at step %d due to reaching max kl.'%i)
            break
    logger.store(StopIter=i)
    for _ in range(train_v_iters):
        sess.run(train_v, feed_dict=inputs)

    # Log changes from update
    pi_l_new, v_l_new, kl, cf = sess.run([pi_loss, v_loss, approx_kl, clipfrac], feed_dict=inputs)
    plots_data["pi_loss"].append(pi_l_old)
    plots_data["v_loss"].append(v_l_old)
    logger.store(LossPi=pi_l_old, LossV=v_l_old, 
                 KL=kl, Entropy=ent, ClipFrac=cf,
                 DeltaLossPi=(pi_l_new - pi_l_old),
                 DeltaLossV=(v_l_new - v_l_old))

In [ ]:
start_time = time.time()
o, r, d, ep_ret, ep_len = env.reset(), 0, False, 0, 0

# Main loop: collect experience in env and update/log each epoch
for epoch in tqdm_notebook(range(epochs)):
    for t in range(local_steps_per_epoch):
        a, v_t, logp_t = sess.run(get_action_ops, feed_dict={x_ph: o.reshape(1,-1)})

        # save and log
        buf.store(o, a, r, v_t, logp_t)
        logger.store(VVals=v_t)

        o, r, d, _ = env.step(a[0])

        ep_ret += r
        ep_len += 1

        terminal = d or (ep_len == max_ep_len)
        if terminal or (t==local_steps_per_epoch-1):
            if not(terminal):
                print('Warning: trajectory cut off by epoch at %d steps.'%ep_len)
            # if trajectory didn't reach terminal state, bootstrap value target
            last_val = r if d else sess.run(v, feed_dict={x_ph: o.reshape(1,-1)})
            buf.finish_path(last_val)
            if terminal:
                # only save EpRet / EpLen if trajectory finished
                plots_data["rewards"].append(ep_ret)
                logger.store(EpRet=ep_ret, EpLen=ep_len)
            o, r, d, ep_ret, ep_len = env.reset(), 0, False, 0, 0

    # Save model
    if (epoch % save_freq == 0) or (epoch == epochs-1):
        logger.save_state({'env': env}, None)

        saver = tf.train.Saver()
        save_path = saver.save(sess, f'./models/PPO/{env_name}/"SE"-epoch-{epoch}.ckpt')
        print("Model saved in path: %s" % save_path)

    # Perform PPO update!
    update()
    
    if epoch % log_info_every_epoch == 0:
        # Log info about epoch
        logger.log_tabular('Epoch', epoch)
        logger.log_tabular('EpRet', with_min_and_max=True)
        logger.log_tabular('EpLen', average_only=True)
        logger.log_tabular('VVals', with_min_and_max=True)
        logger.log_tabular('TotalEnvInteracts', (epoch+1)*steps_per_epoch)
        logger.log_tabular('LossPi', average_only=True)
        logger.log_tabular('LossV', average_only=True)
        logger.log_tabular('DeltaLossPi', average_only=True)
        logger.log_tabular('DeltaLossV', average_only=True)
        logger.log_tabular('Entropy', average_only=True)
        logger.log_tabular('KL', average_only=True)
        logger.log_tabular('ClipFrac', average_only=True)
        logger.log_tabular('StopIter', average_only=True)
        logger.log_tabular('Time', time.time()-start_time)
        logger.dump_tabular()

W0723 20:59:47.137800 140397783583616 deprecation.py:323] From /home/suriknik/Projects/sirius/mujoco/logx.py:226: simple_save (from tensorflow.python.saved_model.simple_save) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
W0723 20:59:47.139538 140397783583616 deprecation.py:323] From /home/suriknik/anaconda3/lib/python3.7/site-packages/tensorflow/python/saved_model/signature_def_utils_impl.py:201: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


Model saved in path: ./models/PPO/SpiderEnvMany2/"SE"-epoch-0.ckpt
Early stopping at step 1 due to reaching max kl.
---------------------------------------
|             Epoch |               0 |
|      AverageEpRet |            67.4 |
|          StdEpRet |            1.43 |
|          MaxEpRet |              70 |
|          MinEpRet |            64.1 |
|             EpLen |             130 |
|      AverageVVals |          -0.415 |
|          StdVVals |           0.273 |
|          MaxVVals |           0.912 |
|          MinVVals |          -0.936 |
| TotalEnvInteracts |           4e+03 |
|            LossPi |       -2.93e-08 |
|             LossV |             691 |
|       DeltaLossPi |          0.0521 |
|        DeltaLossV |            -350 |
|           Entropy |            16.6 |
|                KL |           0.156 |
|          ClipFrac |            0.66 |
|          StopIter |               1 |
|              Time |            26.6 |
---------------------------------------
Earl

In [ ]:
# Moving average
def smoothen_curve(points, factor=0.9):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points


def plot(array, title='', axis=plt):
    axis.title.set_text(title)
    axis.plot(np.arange(len(array)), smoothen_curve(array))

In [ ]:
%matplotlib inline
#************ Plotting debug info ****************
fig, axs = plt.subplots(2, 3, figsize=(16, 9))
plot(buf.rew_buf, "Rewards", axs[0, 0])
plot(plots_data["v_loss"], "Critic loss", axs[0, 1])
plot(buf.adv_buf, "Advantage mean", axs[0, 2])
plot(plots_data["pi_loss"], "Actor loss", axs[1, 0])
plot(buf.act_buf.max(axis=1), "Max action", axs[1,1])
plt.suptitle(env_name, fontsize=18)
plt.savefig("./models/PPO/{}/plots/reward({}).png".format(env_name, int(np.array(buf.rew_buf[-10:]).mean())))
#*************************************************

In [ ]:
stop

In [ ]:
env = SpiderEnvMany2()
obs = env.reset()
while True:
    try:
        ac, _, _ = sess.run(get_action_ops, feed_dict={x_ph: obs.reshape(1,-1)})

        obs, reward, done, _ = env.step(ac[0])
        env.render()
    except KeyboardInterrupt:
        env.close()
        break
if done:
    env.reset()